In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import csv


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import csv
from urllib.parse import urljoin

# Specify the path to the ChromeDriver executable
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("executable_path=chromedriver.exe")

# Initialize the Chrome WebDriver with the options
browser = webdriver.Chrome(options=chrome_options)

url = 'https://www.sulekha.com/overseas-education-consultants/delhi'
browser.get(url)

fields = ['Name', 'Phone', 'Address', 'Verified', 'Premium']
out_file = open('sulekha.csv', 'w', newline='', encoding='utf-8')  # Added newline and encoding
csvwriter = csv.DictWriter(out_file, delimiter=',', fieldnames=fields)
csvwriter.writeheader()  # Write the header row

visited_entries = set()  # Initialize a set to track visited entries

while True:
    # Check for the presence of a pop-up
    try:
        close_button = browser.find_element(By.CLASS_NAME, 'close.filled svg')
        if close_button:
            # If the close button is found, click it using JavaScript
            browser.execute_script("arguments[0].click();", close_button)
            time.sleep(2)  # Wait for the pop-up to close
    except:
        pass  # Ignore any exceptions related to pop-up handling

    # Get the page source
    source = browser.page_source
    soup = BeautifulSoup(source, 'lxml')
    # Find all the <div> elements with class "name"
    div_elements = soup.find_all('div', class_='name')

    for div_element in div_elements:
        # Extract the <a> element with the specific structure
        a_element = div_element.find('a', href=True)

        if a_element:
            # Extract the href from the <a> element (relative path)
            relative_url = a_element['href']

            if relative_url not in visited_entries:
                # If this entry has not been visited, scrape it
                visited_entries.add(relative_url)

                # Construct the absolute URL
                new_tab_url = urljoin(url, relative_url)

                # Open the new tab using JavaScript
                browser.get(new_tab_url)

                # Wait for the page to load (5 seconds)
                time.sleep(5)

                # Extract Name, Phone, and Address from the new tab
                source = browser.page_source
                soup = BeautifulSoup(source, 'lxml')

                name_element = soup.find('h1', class_='title-primary')
                name = name_element.text if name_element else 'Name not found'

                # Extract the phone number using the specific sulga attribute
                phone_element = soup.find('a', href=True, sulga="Profile - Icon Click First Fold^Call Click^Call Button")
                phone = phone_element['href'] if phone_element else 'Phone not found'

                # Extract the address using a specific condition
                address_element = soup.select_one('.item .title-medium:-soup-contains("Address") + p')
                address = address_element.text if address_element else 'Address not found'

                # Extract the SVG elements for "Verified" and "Premium" badges
                verified_badge = soup.find('svg', class_='verified-tick')
                premium_badge = soup.find('svg', class_='premium-badge')

                # Check if the "Verified" badge is present
                verified_status = 'Verified' if verified_badge else 'Unverified'

                # Check if the "Premium" badge is present
                premium_status = 'Premium' if premium_badge else 'N/A'

                # Update the 'details' dictionary with the badge statuses
                details = {'Name': name, 'Phone': phone, 'Address': address, 'Verified': verified_status, 'Premium': premium_status}

                try:
                    csvwriter.writerow(details)
                    out_file.flush()  # Flush the data to the file
                    print(details)
                except:
                    print('Error in writing')

                # Click the back button to return to the original URL
                browser.get(url)  # Go back to the original URL

    # Check if there's a "View More" button to load more entries
    view_more_button = browser.find_elements(By.XPATH, "//span[@class='button line-primary medium' and @id='btnviewmorebiz']")
    if view_more_button:
        view_more_button[0].click()
        time.sleep(3)  # Wait for the next set of entries to load
    else:
        break  # No more entries, exit the loop

out_file.close()  # Close the CSV file when done


The chromedriver version (119.0.6045.105) detected in PATH at C:\Users\ssamy\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (120.0.6099.129); currently, chromedriver 120.0.6099.109 is recommended for chrome 120.*, so it is advised to delete the driver in PATH and retry


{'Name': 'Inspired Overseas Janakpuri Delhi', 'Phone': 'tel:08048046332', 'Address': '701-702, Plot No. 10, Vishal Tower, District Centre, Janakpuri, Delhi - 110058', 'Verified': 'Verified', 'Premium': 'Premium'}
{'Name': 'Mindways Consulting Nehru Place Delhi', 'Phone': 'tel:08069876334', 'Address': '1106, 11th Floor, Chiranjiv Tower 43, Nehru Place, Delhi - 110019', 'Verified': 'Verified', 'Premium': 'Premium'}
{'Name': 'AB Educational Avenues Pvt. Ltd. Nehru Place Delhi', 'Phone': 'tel:08069873650', 'Address': 'No. 103, Chiranjivi Tower, Nehru Place, Delhi - 110019', 'Verified': 'Verified', 'Premium': 'Premium'}
{'Name': 'Keemaya Overseas Education Services Dwarka Delhi', 'Phone': 'tel:08069876047', 'Address': 'No. D-484, 3rd Floor, Ramphal Chowk, Sec-7, Dwarka, Delhi - 110075', 'Verified': 'Verified', 'Premium': 'Premium'}
{'Name': 'Academic & Professional Studies Abroad - APSA Sector 53 Gurgaon', 'Phone': 'tel:08069868522', 'Address': 'No. 1309, 13th Floor, Devika Tower 6, Nehru P